In [2]:
import datetime as dt
import pandas as pd
import os
import requests
import cv2

In [ ]:
download_dir = '/data2/elilouis/sublimationofsnow/eol_photos/'

In [ ]:
start_date = '20230501'
end_date = '20230525'
DATE_FORMAT_STR = '%Y%m%d'

In [ ]:
def get_url(date):
    return \
    f"https://archive.eol.ucar.edu/docs/isf/projects/SOS/isfs/qcdata/images/{date}/surface.ISFS.{date}0002.photo.jpg"

def download(url: str, dest_folder: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))


In [ ]:
datelist = pd.date_range(
    dt.datetime.strptime(start_date, DATE_FORMAT_STR),
    dt.datetime.strptime(end_date, DATE_FORMAT_STR),
    freq='d'
).strftime(DATE_FORMAT_STR).tolist()


In [ ]:
for date in datelist:
    print(f"downloading: {date}")
    download(
        get_url(date),
        download_dir
    )
    assert os.path.isfile(
        os.path.join(download_dir, f"surface.ISFS.{date}0002.photo.jpg")
    )

In [ ]:

import os

image_folder = 'images'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()
